In [231]:
import os
import copy
import json
import re

import pandas as pd
import numpy as np

import PyPDF2
import gender_guesser.detector as gender

# import pyproj # convert lat/lon into Plotly standard

# import plotly.express as px
# import plotly.graph_objects as go

pd.set_option('display.max_columns', 70)

# import plotly.offline as pyo
# pyo.init_notebook_mode(connected=True)
# import plotly.io as pio
# pio.renderers.default = "vscode"


In [232]:
PWD = os.getcwd()
DATA_PATH = PWD + "/data/"

FY_DATA_PATH = DATA_PATH + "financial-statements/"
FY_BASE_NAME = "UBC-FY"

## Manually Set page ranges



In [233]:
page_limits = {
    '2021':{'min':42, 'max':141},
    '2022':{'min':45, 'max':165},
    '2023':{'min':44, 'max':172},
    '2024':{'min':43, 'max':188},}

In [234]:

def null_str_to_nan(test_number):

    if test_number.isnumeric():
        return test_number
    else:
        return np.nan
    

def is_numeric(test_string):

    return test_string.lstrip('-').replace(',','').replace("(", "").replace(")", "").isnumeric()





def pdf_to_df(pdf_file, year, page_start, page_end):

    pdfFileObj = open(pdf_file, 'rb')
    
    # creating a pdf reader object
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    
    column_header_pattern = r"Name\s*Remuneration\s*Expenses"

    data = pd.DataFrame({'name':[], 'year':[], 'salary':[], 'expense':[]})

    for page in np.arange(page_start, page_end, 1):

        # print('+'*80)

        page = int(page)
        pageObj = pdfReader.getPage(page)
        pagetxt = pageObj.extractText()


        begin = 0
        finished_prof = 0
        prof = []

        for index, line in enumerate(pagetxt.split('\n')):

        
            if begin == 1:

        
                if finished_prof == 1:
                    name = ''
                    pay = null_str_to_nan(prof[-2].replace(',','').replace("(", "").replace(")", ""))
                    exp = null_str_to_nan(prof[-1].replace(',','').replace("(", "").replace(")", ""))
                    for n in prof[0:-2]:
                        name = name + n.upper() + ' '
                    new_row = pd.DataFrame({'name':[name], 'year':[int(year)], 'salary':[pay], 'expense':[exp]})
                    data = pd.concat([data, new_row], ignore_index=True)
                    
                    # print(new_row)
                    # print('-'*80)
                
                    
                    prof = []
                    finished_prof = 0
                    
                if len(line.split()) != 0:# and not re.search(column_header_pattern, line, re.DOTALL):# i.split() != ['Remuneration', 'Expenses*']:                        

                    cleaned_line = re.sub(column_header_pattern, "", line).replace("*", "").strip()

                    # print(cleaned_line)

                    if finished_prof == 0 and (cleaned_line.split()[-1] == '-' or is_numeric(cleaned_line.split()[-1])) and "$" not in cleaned_line: 
                        for k in cleaned_line.split():
                            prof.append(k)
                        finished_prof = 1
                        
        
                    else:
                        for k in line.split():
                            prof.append(k)               
                        finished_prof = 0
                    
                
                # print(line)
                
                        
            
            if re.search(column_header_pattern, line, re.DOTALL):

                begin = 1
        



    # closing the pdf file object
    pdfFileObj.close()


    return data


In [235]:
df = pd.DataFrame({'name':[], 'year':[], 'salary':[], 'expense':[]})

for i, (year, value) in enumerate(page_limits.items()):

    if year  != '2024':
        print(f'Parsing Year {year}')

        pdf_file = FY_DATA_PATH + FY_BASE_NAME + year + '.pdf'
        page_start = value['min']
        page_end = value['max']

        new_df = pdf_to_df(pdf_file, int(year), page_start, page_end)

        df = pd.concat([df, new_df], ignore_index=True)


Parsing Year 2021
Parsing Year 2022
Parsing Year 2023


In [236]:
# df

### Checking for people with the same name

In [237]:
# Count occurrences of each name within each year
name_counts = df.groupby(["year", "name"]).size().reset_index(name="count")

# Merge counts back to the original DataFrame
df = df.merge(name_counts, on=["year", "name"], how="left")

# Assign numbers only to names that appear more than once
df["name_id"] = df.groupby(["year", "name"]).cumcount() + 1  # Start numbering from 1
df["name_id"] = df.apply(
    lambda x: f"{x['name']} {x['name_id']}" if x["count"] > 1 else x["name"],
    axis=1
)

# Drop the auxiliary count column (optional)
df.drop(columns=["count"], inplace=True)


rows_with_numbers = df[df["name_id"].str.contains(r"\d", na=False)]


# check the duplicate occurance of names to see if it matches the sheets
# print(rows_with_numbers.to_string())


### Work out if male or female

In [238]:
unique_names = df["name"].unique()


def most_common_gender(gender_list):
    # Initialize counts for 'male' and 'female'
    male_count = 0
    female_count = 0

    # Count occurrences of 'male' and 'female', ignoring 'unknown'
    for gender in gender_list:
        if gender == 'male':
            male_count += 1
        elif gender == 'female':
            female_count += 1

    # Determine which gender has the most occurrences
    if male_count > female_count:
        return 'male'
    elif female_count > male_count:
        return 'female'
    else:
        return 'unknown'  # Return None if both are equal or the list has no valid entries



# Pass the entire full name to the detector
d = gender.Detector()

# Function to process and estimate gender
def estimate_gender(full_name):
    # Normalize the name (capitalize each word for consistent processing)
    full_name = full_name.title()  # "CARROLL, MICHAEL" -> "Carroll, Michael"
    split_name = full_name.split(',')
    first_name = split_name[-1].strip().title()

    temp_gender = []

    for split_first_name in first_name.split(' '):

        split_first_name = split_first_name.strip().title()
        temp_gender.append(d.get_gender(split_first_name))


    guessed_gender = most_common_gender(temp_gender)

    # if guessed_gender == 'unknown':
        # print(full_name)
        # full_name = full_name.split(',')[-2].strip().title()
        # guessed_gender = d.get_gender(full_name)

    return guessed_gender



# Apply the function to the list of names
results = [(name, estimate_gender(name)) for name in unique_names]

name_gender_dict = dict(results)


In [239]:
# Count occurrences of each gender category
gender_counts = {
    "male": 0,
    "female": 0,
    "unknown": 0
}

for _, gender_detect in results:
    if gender_detect in gender_counts:
        gender_counts[gender_detect] += 1

# Print the counts
print("Counts by Gender:")
for gender_detect, count in gender_counts.items():
    print(f"{gender_detect.capitalize()}: {count}")

Counts by Gender:
Male: 3791
Female: 3586
Unknown: 1686


### Adding sex to the dataframe

In [240]:
df['gender'] = df['name'].map(name_gender_dict)

### Ensuring salary and expenses are numeric

In [241]:
df['salary'] = pd.to_numeric(df['salary'], errors='coerce')
df['expense'] = pd.to_numeric(df['expense'], errors='coerce')

### Save to .csv

In [242]:
csv_file = DATA_PATH + 'output/salary.csv'


df.to_csv(csv_file, index=False) 

---
---

## Creating new csv with calculations

In [243]:
def get_yearly_changes(df, name_id, years):

    number_of_years = len(years)

    year_range, salary_amount_change, salary_percent_change, yearly_check = [], [], [], []
    year_range_str = ''
    salary_amount_change_num, salary_percent_change_num = 0, 0

    if number_of_years <= 1:
        return np.nan, np.nan, np.nan, np.nan
    

    for index, year in enumerate(years):

        filtered_row = df[(df['year'] == year) & (df['name_id'] == name_id)].iloc[0]

        if index + 1 < number_of_years:

            if years[index+1] - year == 1:
                year_range_str = f"{year}-{years[index+1]}"
                
                filtered_row_new = df[(df['year'] == years[index+1]) & (df['name_id'] == name_id)].iloc[0]
                salary_amount_change_num = filtered_row_new['salary'] - filtered_row['salary']
                salary_percent_change_num = (filtered_row_new['salary'] - filtered_row['salary'])/filtered_row['salary']

                year_range.append(year_range_str)
                salary_amount_change.append(salary_amount_change_num)
                salary_percent_change.append(salary_percent_change_num)
                yearly_check.append(True)

                if number_of_years <= 2:
                    break
        else:
            year_range_str = f"{years[0]}-{year}"

            filtered_row_new = df[(df['year'] == years[0]) & (df['name_id'] == name_id)].iloc[0]
            salary_amount_change_num = filtered_row['salary'] - filtered_row_new['salary']
            salary_percent_change_num = (filtered_row['salary'] - filtered_row_new['salary'])/filtered_row_new['salary']

            year_range.append(year_range_str)
            salary_amount_change.append(salary_amount_change_num)
            salary_percent_change.append(salary_percent_change_num)
            yearly_check.append(False)
        

    return year_range, salary_amount_change, salary_percent_change, yearly_check


In [244]:
# unique_names = df["name_id"].unique()
years_by_id = df.groupby('name_id')['year'].apply(list).to_dict()

calc_df = pd.DataFrame({'name_id':[], 'year_range':[], 'salary_amount_change':[], 'salary_percent_change':[], 'yearly_check':[]})


# for unique_name in unique_names:
for index, (name_id, years) in enumerate(years_by_id.items()):

    if len(years) > 1:
        
        year_range, salary_amount_change, salary_percent_change, yearly_check = get_yearly_changes(df, name_id, years)

        # print(name_id, years, year_range, salary_amount_change, yearly_check)   

        # print(f'Name id {index} / {len(years_by_id)}')

        new_row = pd.DataFrame({'name_id':[name_id]*len(year_range), 
                                'year_range':year_range, 
                                'salary_amount_change':salary_amount_change, 
                                'salary_percent_change':salary_percent_change,
                                'yearly_check':yearly_check})
        
        calc_df = pd.concat([calc_df, new_row], ignore_index=True)



### Save to .csv

In [247]:
csv_calcfile = DATA_PATH + 'output/id-calc.csv'


calc_df.to_csv(csv_calcfile, index=False) 

### Look at largest salary increases between years

In [248]:
# calc_df.nlargest(20, 'salary_amount_change')